# ANALYZING RTC SEVERITY DATASET
In this project, we are analyzing rioad accident data in order to answer the following questions:
1. When did most road accidents occur ? 
2. What hour ?
3. Which day had the most/least accidents ?
4. Which month has the most accidents ?
5. Which roads lead to major road accidents.
6. Are there specific locations that are prone to accidents ?
7. Are there more accidents in rural/ urban areas ?
8. Driving at which speed limit leads to accidents ?

In [1]:
import csv
import datetime as dt
import more_itertools
import locale
import chardet

In [2]:
# get the encoding by reading the first ten lines
with open("C://Users//user//Downloads//accident_data.csv//accident_data.csv", mode='rb') as file:
    raw_bytes = file.read(10)
    detected_encoding = chardet.detect(raw_bytes)['encoding']
    print(detected_encoding)

ascii


In [3]:
print(locale.getpreferredencoding())

cp1252


**CP1252** is the encoding used in the file

# **CONVERT THE FILE FROM *ASCII* ENCODING TO *UTF-8* ENCODING**

## To convert a csv file file from one encoding to another:
1. Convert the file from it's current encoding.
2. Read the file using **csv.reader()**
3. Open the new fike using the desired encoding
4. Loop over the rows of the original file and write them into the new one using **csv.writer()** and the **writerow()** method.

In [4]:
with open ("C://Users//user//Downloads//accident_data.csv//accident_data.csv") as file:
    rows = list(csv.reader(file))

In [5]:
print(rows[:5])

[['Accident_Index', '1st_Road_Class', '1st_Road_Number', '2nd_Road_Class', '2nd_Road_Number', 'Accident_Severity', 'Carriageway_Hazards', 'Date', 'Day_of_Week', 'Did_Police_Officer_Attend_Scene_of_Accident', 'Junction_Control', 'Junction_Detail', 'Latitude', 'Light_Conditions', 'Local_Authority_(District)', 'Local_Authority_(Highway)', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'LSOA_of_Accident_Location', 'Number_of_Casualties', 'Number_of_Vehicles', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Police_Force', 'Road_Surface_Conditions', 'Road_Type', 'Special_Conditions_at_Site', 'Speed_limit', 'Time', 'Urban_or_Rural_Area', 'Weather_Conditions', 'Year', 'InScotland'], ['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', '04/01/2005', 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E010

In [6]:
header = rows[0]
print(len(header))

34


In [7]:
data = rows[1:]
print(len(data[0]))

34


# EXPLORE THE DATASET

In [8]:
# function to explore the dataset to find out how many rows and columns there are
def explore_dataset(dataset, start, end, rows_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print("\n")
    if rows_columns:
        print("Number of rows",len(dataset))
        print("Number of columns", len(dataset[0]))

In [9]:
explore_dataset(data, 0,5,True)

['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', '04/01/2005', 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E01002849', '1', '1', '0', '1', 'Metropolitan Police', 'Wet or damp', 'Single carriageway', 'None', '30', '17:42', 'Urban', 'Raining no high winds', '2005', 'No']


['200501BS00002', 'B', '450', 'C', '0', 'Slight', 'None', '05/01/2005', 'Wednesday', '1', 'Auto traffic signal', 'Crossroads', '51.520075', 'Darkness - lights lit', 'Kensington and Chelsea', 'Kensington and Chelsea', '524170', '181650', '-0.211708', 'E01002909', '1', '1', '0', '5', 'Metropolitan Police', 'Dry', 'Dual carriageway', 'None', '30', '17:36', 'Urban', 'Fine no high winds', '2005', 'No']


['200501BS00003', 'C', '0', '', '0', 'Slight', 'None', '06/01/2005', 'Thursday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.525301', 

# CHECK FOR WRONG DATA

In [10]:
# Remove empty lists
data = [sublist for sublist in data if sublist]
explore_dataset(data, 0,5,True)

['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', '04/01/2005', 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E01002849', '1', '1', '0', '1', 'Metropolitan Police', 'Wet or damp', 'Single carriageway', 'None', '30', '17:42', 'Urban', 'Raining no high winds', '2005', 'No']


['200501BS00002', 'B', '450', 'C', '0', 'Slight', 'None', '05/01/2005', 'Wednesday', '1', 'Auto traffic signal', 'Crossroads', '51.520075', 'Darkness - lights lit', 'Kensington and Chelsea', 'Kensington and Chelsea', '524170', '181650', '-0.211708', 'E01002909', '1', '1', '0', '5', 'Metropolitan Police', 'Dry', 'Dual carriageway', 'None', '30', '17:36', 'Urban', 'Fine no high winds', '2005', 'No']


['200501BS00003', 'C', '0', '', '0', 'Slight', 'None', '06/01/2005', 'Thursday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.525301', 

In [11]:
print(len(data[0]))

34


In [12]:
roadtypes = []
for row in data:
    road_type = row[1]
    if road_type not in roadtypes:
        roadtypes.append(road_type)
print(roadtypes)

['A', 'B', 'C', '', 'Motorway', 'A(M)']


# CHECK FOR DUPLICATES

In [13]:
# check for duplicates
duplicate_entries = []
unique_entries = []
for row in data:
    accident_id = row[0]
    if accident_id in unique_entries:
        date_day_dicticate_entries.append(accident_id)
#     else:
#         unique_entries.append(accident_id)
# len_unique_entries = len(unique_entries)
len_duplicate_entries = len(duplicate_entries)
# example_duplicate = duplicate_entries[3:5]
# print("There are", len_unique_entries, "unique entries")
print("There are", len_duplicate_entries, "duplicate entries")
# print(example_duplicate)
#     return len_duplicate_entries, len_unique_entries, example_duplicate
# len_duplicate_entries, len_unique_entries, example_duplicate = check_duplicates(data)
# print(
#     f"Number of duplicate entries {len_duplicate_entries}\n"
#     f"Number of unique entries  {len_unique_entries}\n"
#     f"Examples of duplicate entries {example_duplicate}\n"
#      )

There are 0 duplicate entries


# REPLACE MISSING STRINGS WITH "UNKNOWN DATA"

In [14]:
# # fill in the empty strings with the string(unknown Data)
# def fill_missing_strings(i):
#     for row in data:
#         col = row[i]
#         col = col.title()
#         if not col:
#             col = ""
#         row[i] = col

In [15]:
# for i in range(len(header)):
#     fill_missing_strings(i)
# print(data[2:5])

In [16]:
# get the index of each column
col_index = {}
for i in range(len(header)):
    col_index[header[i]] = i
print(col_index)

{'Accident_Index': 0, '1st_Road_Class': 1, '1st_Road_Number': 2, '2nd_Road_Class': 3, '2nd_Road_Number': 4, 'Accident_Severity': 5, 'Carriageway_Hazards': 6, 'Date': 7, 'Day_of_Week': 8, 'Did_Police_Officer_Attend_Scene_of_Accident': 9, 'Junction_Control': 10, 'Junction_Detail': 11, 'Latitude': 12, 'Light_Conditions': 13, 'Local_Authority_(District)': 14, 'Local_Authority_(Highway)': 15, 'Location_Easting_OSGR': 16, 'Location_Northing_OSGR': 17, 'Longitude': 18, 'LSOA_of_Accident_Location': 19, 'Number_of_Casualties': 20, 'Number_of_Vehicles': 21, 'Pedestrian_Crossing-Human_Control': 22, 'Pedestrian_Crossing-Physical_Facilities': 23, 'Police_Force': 24, 'Road_Surface_Conditions': 25, 'Road_Type': 26, 'Special_Conditions_at_Site': 27, 'Speed_limit': 28, 'Time': 29, 'Urban_or_Rural_Area': 30, 'Weather_Conditions': 31, 'Year': 32, 'InScotland': 33}


In [17]:
for row in data:
    myyear = row[32]
    mydates = row[7]
print(type(mydates))

<class 'str'>


# PARSE STRINGS AS DATES

In [18]:
# parse strings as dates
for row in data:
    mydates = row[7]
    if mydates:
        mydates = dt.datetime.strptime(mydates, "%d/%m/%Y").date()
        row[7] = mydates

    mytime = row[29]
#     replace Unknown Data Time with 00:00
    if mytime:
        mytime = dt.datetime.strptime(mytime, "%H:%M").time()
        row[29] = mytime
    
    myyear = row[32]
    if myyear:
        myyear = dt.datetime.strptime(myyear, "%Y").year
        row[32] = myyear
    
print(type(row[7]))
print(type(row[29]))
print(type(row[32]))

<class 'datetime.date'>
<class 'datetime.time'>
<class 'int'>


In [19]:
explore_dataset(data, 0,5,True)

['200501BS00001', 'A', '3218', '', '0', 'Serious', 'None', datetime.date(2005, 1, 4), 'Tuesday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.489096', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '525680', '178240', '-0.19117', 'E01002849', '1', '1', '0', '1', 'Metropolitan Police', 'Wet or damp', 'Single carriageway', 'None', '30', datetime.time(17, 42), 'Urban', 'Raining no high winds', 2005, 'No']


['200501BS00002', 'B', '450', 'C', '0', 'Slight', 'None', datetime.date(2005, 1, 5), 'Wednesday', '1', 'Auto traffic signal', 'Crossroads', '51.520075', 'Darkness - lights lit', 'Kensington and Chelsea', 'Kensington and Chelsea', '524170', '181650', '-0.211708', 'E01002909', '1', '1', '0', '5', 'Metropolitan Police', 'Dry', 'Dual carriageway', 'None', '30', datetime.time(17, 36), 'Urban', 'Fine no high winds', 2005, 'No']


['200501BS00003', 'C', '0', '', '0', 'Slight', 'None', datetime.date(2005, 1, 6), 'Thursday', '1', 'Data missing or out 

In [20]:
dates_1 = []
day_1 = []
for row in data:    
    mydate = row[7]
    dates_1.append(mydate)
    day_of_week = row[8]
    day_1.append(day_of_week)
print(len(dates_1))
print(len(day_1))

# Using the `zip` function to combine the two lists into a list of tuples
date_data = list(zip(dates_1, day_1))

# Converting the list of tuples to a dictionary
date_dict = dict(date_data)
# print the first few elements of the dictionary 
first_five = dict(list(date_dict.items())[:4])

print(first_five)

# Converting the dictionary to a list of dictionaries
dict_list = [{'Date': key, 'Day Of Week': value} for key, value in date_dict.items()]
print(dict_list[2:5])

1048575
1048575
{datetime.date(2005, 1, 4): 'Tuesday', datetime.date(2005, 1, 5): 'Wednesday', datetime.date(2005, 1, 6): 'Thursday', datetime.date(2005, 1, 7): 'Friday'}
[{'Date': datetime.date(2005, 1, 6), 'Day Of Week': 'Thursday'}, {'Date': datetime.date(2005, 1, 7), 'Day Of Week': 'Friday'}, {'Date': datetime.date(2005, 1, 10), 'Day Of Week': 'Monday'}]


In [21]:
print(data[100:104])

[['200501BS70083', 'B', '450', '', '0', 'Slight', 'None', datetime.date(2005, 3, 17), 'Thursday', '1', 'Data missing or out of range', 'Not at junction or within 20 metres', '51.517263', '', 'Kensington and Chelsea', 'Kensington and Chelsea', '524290', '181340', '-0.210089', 'E01002831', '1', '1', '0', '4', 'Metropolitan Police', 'Dry', 'Single carriageway', 'None', '30', datetime.time(9, 15), 'Urban', 'Fine no high winds', 2005, 'No'], ['200501BS70084', 'B', '450', 'Unclassified', '0', 'Slight', 'None', datetime.date(2005, 3, 13), 'Sunday', '3', 'Give way or uncontrolled', 'Crossroads', '51.516353', 'Darkness - lights lit', 'Kensington and Chelsea', 'Kensington and Chelsea', '524340', '181240', '-0.209404', 'E01002879', '1', '2', '0', '0', 'Metropolitan Police', 'Dry', 'Single carriageway', 'None', '30', datetime.time(0, 15), 'Urban', 'Fine no high winds', 2005, 'No'], ['200501BS70085', 'C', '0', '', '0', 'Slight', 'None', datetime.date(2005, 3, 15), 'Tuesday', '1', 'Data missing or o

# THE MOST AND LEAST ACCIDENTS 

In [30]:
def most_least(data):    
    # which day had the most accidents?
    result_list = []
    for row in data:
        accident_at = row[7]
        no_casualties = int(row[20])
        day_of_week = row[8]
        my_time = row[29]
        result_list.append([accident_at,no_casualties,day_of_week,my_time])

    accidents_by_day = {}
    casualties_by_day = {}
    day_accident = {}
    accidents_by_hour = {}
    casualties_by_hour = {}
    for result in result_list:
        mydates = result[0]
        casualties = result[1]
        day = result[2]
        time = result[3]
        time = time.strftime("%H")
 
        if mydates in accidents_by_day:
            accidents_by_day[mydates] += 1
            casualties_by_day[mydates] += casualties
        else:
            accidents_by_day[mydates] = 1
            casualties_by_day[mydates] = casualties
        if day in day_accident:
            day_accident[day] += 1
        else:
            day_accident[day] = 1
        if time in accidents_by_hour:
            accidents_by_hour[time] += 1
            casualties_by_hour[time] = casualties
        else:
            accidents_by_hour[time] = 1
            casualties_by_hour[time] = casualties
            
    return accidents_by_day, casualties_by_day, day_accident, accidents_by_hour, casualties_by_hour
accidents_by_day, casualties_by_day, day_accident, accidents_by_hour, casualties_by_hour = most_least(data)

def print_first_few_data(dictionary):
    # first 5 key:value pairs
    first_few = more_itertools.take(10, dictionary.items())
    return first_few

def sorted_values(dictionary):
    # sort the dictionary to get it in descending order
    #sort to see the when most accidents occured and the least
    sorted_dict = dict(sorted(dictionary.items(), reverse = True, key=lambda item: item[1]))
    sorted_dict1 = print_first_few_data(sorted_dict)
    return sorted_dict1

AttributeError: 'str' object has no attribute 'strftime'

In [29]:
print("The first ten accidents by date are: \n ", print_first_few_data(accidents_by_day))
print("The first ten casualties by date are: \n ", print_first_few_data(casualties_by_day))
print("The first ten accidents by hour are: \n ", print_first_few_data(accidents_by_hour))
print("The first ten casualties by hour are: \n ", print_first_few_data(casualties_by_hour))
print("The first ten accidents by day in ascending order:\n", sorted_values(accidents_by_day ))
print("The first ten casualties by day in ascending order:\n", sorted_values(casualties_by_day))
print("The first ten accidents by hour in ascending order:\n", sorted_values(accidents_by_hour ))
print("The first ten casualties by hour in ascending order:\n", sorted_values(casualties_by_hour))
print("The number of accidents per day are: \n ", print_first_few_data(day_accident))
print("The sorted number of accidents per day are: \n ", sorted_values(day_accident))

The first ten accidents by date are: 
  [(datetime.date(2005, 1, 4), 473), (datetime.date(2005, 1, 5), 523), (datetime.date(2005, 1, 6), 553), (datetime.date(2005, 1, 7), 510), (datetime.date(2005, 1, 10), 523), (datetime.date(2005, 1, 11), 549), (datetime.date(2005, 1, 13), 657), (datetime.date(2005, 1, 14), 715), (datetime.date(2005, 1, 15), 461), (datetime.date(2005, 1, 16), 412)]
The first ten casualties by date are: 
  [(datetime.date(2005, 1, 4), 599), (datetime.date(2005, 1, 5), 686), (datetime.date(2005, 1, 6), 710), (datetime.date(2005, 1, 7), 656), (datetime.date(2005, 1, 10), 688), (datetime.date(2005, 1, 11), 734), (datetime.date(2005, 1, 13), 900), (datetime.date(2005, 1, 14), 949), (datetime.date(2005, 1, 15), 662), (datetime.date(2005, 1, 16), 610)]
The first ten accidents by hour are: 
  [('15', 1048575)]
The first ten casualties by hour are: 
  [('15', 1)]
The first ten accidents by day in ascending order:
 [(datetime.date(2005, 10, 21), 822), (datetime.date(2005, 11, 

In [ ]:
def minmax(dictionary):
    maxx = None
    minn = None
    for key in dictionary:
        if maxx is None or dictionary[key] > maxx:
            maxx = dictionary[key]
        if minn is None or dictionary[key] < minn:
            minn = dictionary[key]
    highest_period = None
    lowest_period = None
    for key in dictionary:
        if dictionary[key] == maxx:
            highest_period = key
        if dictionary[key] == minn:
            lowest_period = key
    return maxx, minn, highest_period,lowest_period

In [ ]:
maxx, minn, highest_period,lowest_period = minmax(casualties_by_day)
print("The most number of casualties is {} and occured in {}".format(maxx,highest_period))
print("The least number of casualties is {} and occured in {}".format(minn,lowest_period))

In [ ]:
maxx, minn, highest_period,lowest_period = minmax(accidents_by_day)
print("The most number of accidents is {} and occured in {}".format(maxx,highest_period))
print("The least number of accidents is {} and occured in {}".format(minn,lowest_period))

In [ ]:
maxx, minn, highest_period,lowest_period = minmax(day_accident)
print("The most number of accidents is {} and occured on {}".format(maxx,highest_period))
print("The least number of accidents is {} and occured on {}".format(minn,lowest_period))

In [ ]:
maxx, minn, highest_period,lowest_period = minmax(casualties_by_hour)
print("The most number of casualties is {} and occured at the {}th hour".format(maxx,highest_period))
print("The least number of casualties is {} and occured at the {}th hour".format(minn,lowest_period))

In [ ]:
maxx, minn, highest_period,lowest_period = minmax(accidents_by_hour)
print("The most number of accidents is {} and occured at the {}th hour".format(maxx,highest_period))
print("The least number of accidents is {} and occured at the {}th hour".format(minn,lowest_period))

# FINDINGS

1. The most number of casualties is **1157** and occured on **2005-10-21**
2. The least number of casualties is **157** and occured in **2010-01-10**
3. The most number of accidents is **822** and occured on **2005-10-21**
4. The least number of accidents is **118** and occured on **2008-12-25**
5. Most accidents were **171910** which occured on **Friday**
6. The least accidents were **116033** which occured on **Sunday**
7. The most number of casualties by hour is **8** and occured at the **16th** hour
8. The least number of casualties is **1** and occured at the **06** hour
9. The most number of accidents by hour is **91861** and occured at the **17th** hour
10. The least number of accidents by hour is **5789** and occured at the **04th** hour
11. A lot of accidents occur in urban areas, rural areas have got half as much.
12. **Bingham, Nottingham, UK(52.949719, -0.977611)** has the most accidents.
13. Even though **30km/hr** seems ideal to be driving at, most accidents still result from the same
14. There are **138,192** serious accidents, **895,883** slight accidents, **14,500** fatal accidents and **0** unclassified accidents

# DO MOST ACCIDENTS HAPPEN IN RURAL OR URBAN AREAS?

In [ ]:
def my_count(data,index):
    count_dict = {}
    for row in data:
        column = row[index]
        if column not in count_dict:
            count_dict[column] = 1
        else:
            count_dict[column] += 1
    return count_dict

In [ ]:
rural_or_urban = my_count(data,30)
print(sorted_values(rural_or_urban))

In [ ]:
speed_limit = my_count(data,28)
print(sorted_values(speed_limit))

In [ ]:
def accidents_location(data):
    coordinates = {}
    for row in data:
        accident_at = row[7]
        latitude = row[12]
        longitude = row[18]
        coords = (latitude,longitude)
        if coords in coordinates:
            coordinates[coords] += 1
        else:
            coordinates[coords] = 1
    return coordinates
coordinates = accidents_location(data)        

In [ ]:
# which of these combinations have the most accidents?
sorted_values(coordinates)

# THE SPEED LIMIT UNDER WHICH MOST ACCIDENTS OCCURED

In [ ]:
# combine roadclass and road number then compare
full_road1 = []
full_road2 = []
for row in data:
    road_class = row[1]
    road_number = row[2]
    road_class2 = row[3]
    road_number2 = row[4]
    if road_class and road_number:
        full_road1.append(road_class+road_number)
    else:
        full_road.append(None)
    if road_class2 and road_number2:
        full_road2.append(road_class2+road_number2)
    else:
        full_road2.append(None)
print(full_road1[:20])
print(full_road2[:10])
#we can't do much with the roads as there is a lot of missing data

# OUT OF WHAT WE HAVE, WHICH ONES ARE THE SEVERE, FATAL AND SERIOUS ACCIDENTS?

In [ ]:
# empty lists for the three categories of accidents
Serious = []
Slight = []
Fatal = []
Unclassified = []
# iterate through the rows and append to each list accordingly
for row in data:
    severity = row[5]
    if severity.startswith("Serious"):
        Serious.append(row)
    elif severity.startswith("Slight"):
        Slight.append(row)
    elif severity.startswith("Fatal"):
        Fatal.append(row)
    else:
        Unclassified.append(row)
print("There are {0:,} serious accidents".format(len(Serious)))      
print("There are {0:,} slight accidents".format(len(Slight)))
print("There are {0:,} fatal accidents".format(len(Fatal)))
print("There are {} unclassified accidents".format(len(Unclassified)))

# ANALYSIS OF FATAL ACCIDENTS

In [ ]:
accidents_by_day, casualties_by_day, day_accident, accidents_by_hour, casualties_by_hour=most_least(Fatal)

In [ ]:
print("The first ten fatal accidents by date are: \n ", print_first_few_data(accidents_by_day))
print("The first ten casualties from fatal accidents by date are: \n ", print_first_few_data(casualties_by_day))
print("The first ten fatal accidents by hour are: \n ", print_first_few_data(accidents_by_hour))
print("The first ten casualties from fata l accidents by hour are: \n ", print_first_few_data(casualties_by_hour))
print("The first ten fatal accidents by day in ascending order:\n", sorted_values(accidents_by_day ))
print("The first ten casualties from fatal accidents by day in ascending order:\n", sorted_values(casualties_by_day))
print("The first ten fatal accidents by hour in ascending order:\n", sorted_values(accidents_by_hour ))
print("The first ten casualties from fatal accidents by hour in ascending order:\n", sorted_values(casualties_by_hour))
print("The number of fatal accidents per day are: \n ", print_first_few_data(day_accident))
print("The sorted number of fatal accidents per day are: \n ", sorted_values(day_accident))

In [ ]:
accidents_by_day, casualties_by_day, day_accident, accidents_by_hour, casualties_by_hour=most_least(Serious)

In [ ]:
print("The first ten serious accidents by date are: \n ", print_first_few_data(accidents_by_day))
print("The first ten casualties from serious accidents by date are: \n ", print_first_few_data(casualties_by_day))
print("The first ten serious accidents by hour are: \n ", print_first_few_data(accidents_by_hour))
print("The first ten casualties from serious accidents by hour are: \n ", print_first_few_data(casualties_by_hour))
print("The first ten serious accidents by day in ascending order:\n", sorted_values(accidents_by_day ))
print("The first ten casualties from serious accidents by day in ascending order:\n", sorted_values(casualties_by_day))
print("The first ten serious accidents by hour in ascending order:\n", sorted_values(accidents_by_hour ))
print("The first ten casualties from serious accidents by hour in ascending order:\n", sorted_values(casualties_by_hour))
print("The number of serious accidents per day are: \n ", print_first_few_data(day_accident))
print("The sorted number of serious accidents per day are: \n ", sorted_values(day_accident))

In [ ]:
accidents_by_day, casualties_by_day, day_accident, accidents_by_hour, casualties_by_hour=most_least(Slight)

In [ ]:
print("The first ten slight accidents by date are: \n ", print_first_few_data(accidents_by_day))
print("The first ten casualties from slight accidents by date are: \n ", print_first_few_data(casualties_by_day))
print("The first ten slight accidents by hour are: \n ", print_first_few_data(accidents_by_hour))
print("The first ten casualties from slight accidents by hour are: \n ", print_first_few_data(casualties_by_hour))
print("The first ten slight accidents by day in ascending order:\n", sorted_values(accidents_by_day ))
print("The first ten casualties from slight accidents by day in ascending order:\n", sorted_values(casualties_by_day))
print("The first ten slight accidents by hour in ascending order:\n", sorted_values(accidents_by_hour ))
print("The first ten casualties from slight accidents by hour in ascending order:\n", sorted_values(casualties_by_hour))
print("The number of slight accidents per day are: \n ", print_first_few_data(day_accident))
print("The sorted number of slight accidents per day are: \n ", sorted_values(day_accident))

# CHECK FOR THE CORRECT DATATYPES

In [ ]:
def checktypes(data,header):    
    # Initialize a dictionary to store the data types for each column
    column_data_types = {col: None for col in header}

    # Iterate over each row of the CSV file
    for row in data:
        # Iterate over each column in the row
        for i, col_value in enumerate(row):
            # Check if the data type for the column has been set yet
            if not column_data_types[header[i]]:
                # If not, set the data type to the type of the current value
                column_data_types[header[i]] = type(col_value)
            else:
                # If it has been set, check if the current value has a different data type
                if column_data_types[header[i]] != type(col_value):
                    # If it does, set the data type to a generic "object" type
                    column_data_types[header[i]] = object

    # Print the data types for each column
    for col, data_type in column_data_types.items():
        print(f"{col}: {data_type.__name__}")


In [ ]:
checktypes(data,header)

In [ ]:
# create a dictionary to check for the count of values
def unique_count(data,index):
    unique_dict = {}
    for row in data:
        column = row[index]
        if column not in unique_dict:
            unique_dict[column] = 1
        else:
            unique_dict[column] += 1
    return unique_dict

In [ ]:
mydict = unique_count(data,20)
print(mydict)

In [ ]:
def dict_sum(dict):
    total = 0
    for key in dict:
        total += dict[key]
    return total

In [ ]:
print(dict_sum(mydict))

# FINDINGS
Columns that have str type yet are integers:
1. 1st_Road_Number
2. 2nd_Road_Number
3. Did_Police_Officer_Attend_Scene_of_Accident
4. Latitude
5. Longitude
6. Location_Easting_OSGR
7. Location_Northing_OSGR
8. Number of Vehicles
9. Number of casualties
10. Pedestrian_Crossing-Human_Control
11. Pedestrian_Crossing-Physical_Facilities
12. Speed_limit

# CONVERT STRING DATA TYPES TO INT/FLOAT 

In [ ]:
# list of columns with wrong data type
non_strings_int = ['1st_Road_Number', '2nd_Road_Number','Did_Police_Officer_Attend_Scene_of_Accident',
            'Location_Easting_OSGR','Location_Northing_OSGR','Number_of_Vehicles','Number_of_Casualties',
            'Pedestrian_Crossing-Human_Control','Speed_limit','Pedestrian_Crossing-Physical_Facilities']
non_strings_float = ['Latitude','Longitude']

In [ ]:
print(len(non_strings_int))
print(len(non_strings_float))

In [ ]:
# get the column indices of the column with the wrong data
def get_index(header, mylist):
    cols_convert = []
    for j in range(len(header)):
        for i in range(len(mylist)):
            if mylist[i] == header[j]:
                cols_convert.append(j)
    return cols_convert

In [ ]:
int_columns_to_convert = get_index(header,non_strings_int)
float_columns_to_convert = get_index(header,non_strings_float)
print(int_columns_to_convert)
print(float_columns_to_convert)

In [ ]:
print(row[1])

In [ ]:
# def convert(data,header,alist):
#     converted_data = []
#     # Iterate through each row in the CSV file
#     for row in data:
#         # Convert the specified columns to int
#         for i in alist:
            
#             if row[i] != "Unknown Data" and row[i] != "Na":
#                 row[col_idx] = int(row[col_idx])
#         # Add the converted row to the list of converted data
#         converted_data.append(row)

#     # Write the converted data to a new CSV file
#     with open('converted_example.csv', 'w', newline='') as csvfile:

#         # Create a CSV writer object
#         writer = csv.writer(csvfile)

#         # Write the header row
#         writer.writerow(header)

#         # Write the converted data
#         writer.writerows(converted_data)

In [ ]:
# convert(data,header,int_columns_to_convert)

In [ ]:
# def convert(data,header,mylist):
#     converted_data = []
#     # Iterate through each row in the CSV file
#     for row in data:
#         # Convert the specified columns to int
#         for col_idx in mylist:
#             if row[col_idx] != "Unknown Data" and row[col_idx] != "Na":
#                 row[col_idx] = float(row[col_idx])
#         # Add the converted row to the list of converted data
#         converted_data.append(row)

#     # Write the converted data to a new CSV file
#     with open('converted_example.csv', 'w', newline='') as csvfile:

#         # Create a CSV writer object
#         writer = csv.writer(csvfile)

#         # Write the header row
#         writer.writerow(header)

#         # Write the converted data
#         writer.writerows(converted_data)

In [ ]:
# convert(data,header,float_columns_to_convert)

In [ ]:
# # read the new csv file 
# with open("converted_example.csv") as file:
#     new_data = list(csv.reader(file))
#     header = new_data[0]
#     new_rows = new_data[1:]

In [ ]:
# explore_dataset(new_data, 0,5,True)

In [ ]:
# checktypes(new_data,header)